In [28]:
#Script para gerar Tabela Cliente

import pandas as pd
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import random
import pyodbc
from datetime import date
from faker import Faker

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"
#user = ""
#senha = ""

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
#conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

quant_funcionarios_gerados = 500

#Listas 
nomes = [
    "Maria", "Jose", "Ana", "Joao", "Antonio", "Francisco", "Carlos", "Paulo", "Pedro", "Lucas",
    "Luiz", "Marcos", "Luis", "Rafael", "Gabriel", "Daniel", "Marcelo", "Bruno", "Eduardo", "Felipe",
    "Raimundo", "Miguel", "Arthur", "Gustavo", "Leonardo", "Thiago", "Matheus", "Enzo", "Guilherme", "Davi",
    "Larissa", "Amanda", "Juliana", "Patricia", "Aline", "Camila", "Bruna", "Fernanda", "Isabela", "Mariana",
    "Tatiana", "Vanessa", "Carolina", "Bianca", "Clara", "Vitoria", "Leticia", "Laura", "Beatriz", "Julia",
    "Eliel", "Bernado", "Debora", "Shauanne", "Valdemira", "Rogeria", "Victor", "Paula", "Leandra", "Antonia",
    "Luiza", "Rafaela", "Gabriela", "Daniela", "Marcela", "Eduarda"
]

sobrenomes = [
    "Silva", "Santos", "Oliveira", "Souza", "Rodrigues", "Ferreira", "Alves", "Pereira", "Lima", "Gomes",
    "Costa", "Ribeiro", "Martins", "Carvalho", "Araujo", "Melo", "Barbosa", "Monteiro", "Cardoso", "Teixeira",
    "Mendes", "Nascimento", "Cavalcanti", "Dias", "Castro", "Campos", "Bezerra", "Fernandes", "Vieira", "Correia",
    "Sousa", "Machado", "Rocha", "Neves", "Nunes", "Moreira", "Medeiros", "Freitas", "Barros", "Pinto",
    "Andrade", "Moura", "Dantas", "Cunha", "Brito", "Leal", "Farias", "Peixoto", "Fonseca", "Azevedo"    
]

nomes_ruas = [
    "Rua Sao Joao", "Rua Santa Maria", "Rua Sao Paulo", "Rua Sao Pedro", "Rua Sao Jose",
    "Rua Sao Francisco", "Rua Santo Antonio", "Rua Sao Sebastiao", "Rua Santa Catarina",
    "Rua Luzia", "Rua Santa Rita", "Rua Santa Ana", "Rua Santa Terezinha", "Rua Santa Clara",
    "Rua Isabel", "Rua Santa Helena", "Rua Santa Cecilia", "Rua Santa Rosa", "Rua Santa Monica",
    "Rua Edwiges", "Rua Santa Ines", "Rua Santa Fe", "Rua Santa Lucia", "Rua Santa Efigenia",
    "Rua Barbara", "Rua Santa Angela", "Rua Santa Gertrudes", "Rua Santa Margarida",
    "Rua Marina", "Rua Santa Elisa", "Rua Santa Julia", "Rua Santa Zelia", "Rua Santa Zilda",
    "Rua Zulmira", "Rua Santa Zulene, Rua das Flores", "Rua da Paz", "Rua Sao Joao", "Rua Sao Paulo", "Rua Santo Antonio",
    "Rua 7 de Setembro", "Rua XV de Novembro", "Rua do Comercio", "Rua das Acacias", "Rua das Palmeiras",
    "Rua dos Cravos", "Rua do Sol", "Rua Lua Nova", "Rua da Esperanca", "Rua Sao Pedro",
    "Rua das Orquideas", "Rua das Rosas", "Rua Primavera", "Rua Santa Luzia", "Rua Sao Francisco",
    "Rua das Margaridas", "Rua do Bosque", "Rua das Aguas", "Rua Bela Vista", "Rua Santa Catarina",
    "Rua Dom Pedro I", "Rua Dom Pedro II", "Rua Marechal Deodoro", "Rua da Liberdade", "Rua do Lago",
    "Rua dos Girassois", "Rua Antonio Carlos", "Rua da Independencia", "Rua Presidente Vargas", "Rua das Laranjeiras",
    "Rua do Porto", "Rua Santos Dumont", "Rua Carlos Gomes", "Rua da Vitoria", "Rua do Carmo",
    "Rua do Campo", "Rua da Saudade", "Rua do Progresso", "Rua dos Bandeirantes", "Rua Coronel Antonio",
    "Rua Sao Jose", "Rua Benedito Calixto", "Rua dos Lirios", "Rua Sete Lagoas", "Rua Pedro Alvares Cabral",
    "Rua Castro Alves", "Rua Padre Anchieta", "Rua Machado de Assis", "Rua Monte Castelo", "Rua Humberto de Campos",
    "Rua dos Tucanos", "Rua das Violetas", "Rua do Ipe", "Rua dos Pinheiros", "Rua Jose Bonifacio",
    "Rua Getulio Vargas", "Rua Marechal Rondon", "Rua Visconde de Maua", "Rua do Horizonte", "Rua do Contorno",
    "Rua Rui Barbosa", "Rua Juscelino Kubitschek", "Rua Presidente Joao Goulart", "Rua General Osorio", "Rua dos Sonhos",
    "Rua das Figueiras", "Rua Goias", "Rua Alvorada", "Rua Bela Cintra", "Rua Piaui",
    "Rua Domingos Jorge Velho", "Rua General Carneiro", "Rua Dom Bosco", "Rua Miguel Couto", "Rua Alfredo Pujol",
    "Rua Frei Caneca", "Rua Dr. Arnaldo", "Rua Diogo Feijo", "Rua Regente Feijo", "Rua Conselheiro Lafaiete",
    "Rua Barao de Maua", "Rua Benjamin Constant", "Rua Santa Clara", "Rua Maranhao", "Rua Bahia",
    "Rua Amazonas", "Rua Acre", "Rua Sao Joaquim", "Rua dos Cedros", "Rua Tamoios",
    "Rua Jose de Alencar", "Rua Ceara", "Rua Camelia", "Rua das Cerejeiras", "Rua Japao"]

nomes_bairros = [
    "Centro", "Jardim", "Vila", "Sao Joao", "Sao Paulo", "Sao Pedro", "Sao Jose", "Sao Francisco",
    "Santo Antonio", "Sao Sebastiao", "Santa Maria", "Santa Catarina", "Santa Luzia", "Santa Rita",
    "Santa Ana", "Santa Terezinha", "Santa Clara", "Santa Isabel", "Santa Helena", "Santa Cecilia",
    "Santa Rosa", "Santa Monica", "Santa Edwiges", "Santa Ines", "Santa Fe", "Santa Lucia",
    "Santa Efigenia", "Santa Barbara", "Santa Angela", "Centro", "Jardim das Flores", "Vila Nova", "Jardim America", "Jardim Paulista",
    "Jardim Primavera", "Jardim Europa", "Jardim dos Ipes", "Jardim das Palmeiras", "Jardim Bela Vista",
    "Parque Industrial", "Jardim Alvorada", "Jardim Santa Clara", "Vila Esperanca", "Vila Maria",
    "Parque Sao Jorge", "Jardim Sao Francisco", "Jardim Santa Rita", "Vila Rica", "Vila Progresso",
    "Vila Matilde", "Vila Boa Vista", "Vila Sao Jose", "Parque dos Passaros", "Jardim Imperial",
    "Jardim Sao Luiz", "Jardim das Acacias", "Jardim Mariana", "Jardim do Lago", "Parque das Nacoes",
    "Parque dos Bandeirantes", "Jardim Atlantico", "Jardim Monte Castelo", "Jardim Eldorado", "Jardim Gloria",
    "Vila Oliveira", "Jardim Panorama", "Jardim Dom Bosco", "Vila Uniao", "Jardim Vitoria",
    "Jardim Novo Horizonte", "Vila Industrial", "Vila Augusta", "Jardim Botanico", "Jardim Sao Carlos",
    "Jardim Canada", "Jardim Santa Fe", "Jardim das Hortensias", "Parque Sao Bento", "Jardim dos Pinheiros",
    "Parque do Carmo", "Jardim Sao Joao", "Jardim Maria Helena", "Jardim Santa Rosa", "Jardim Felicidade",
    "Jardim Aurora", "Vila Santana", "Jardim Bela Vista", "Jardim das Magnolias", "Vila Medeiros",
    "Jardim dos Lirios", "Parque das Arvores", "Jardim Santa Terezinha", "Vila Nova Esperanca", "Jardim Santo Antonio",
    "Jardim Independencia", "Jardim Sao Jorge", "Jardim das Oliveiras", "Parque Estrela Dalva", "Jardim Mirante",
    "Vila Sao Pedro", "Parque Bela Vista", "Vila Nova Cachoeirinha", "Jardim Monte Alegre", "Parque Imperial",
    "Jardim Tropical", "Vila Santa Isabel", "Jardim Italia", "Jardim dos Cedros", "Jardim das Nacoes",
    "Vila Real", "Jardim dos Jacarandas", "Jardim Primavera", "Parque Paraiso", "Jardim Universo",
    "Jardim Santo Andre", "Jardim das Figueiras", "Vila Germanica", "Jardim Luz", "Vila Macedo",
    "Jardim Bonfim", "Jardim Colonial", "Jardim das Tulipas", "Vila Curuca", "Jardim Santa Maria"
]

cargos = [
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor','Repositor',
'Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa',
'Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa',
'Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa','Operador de caixa',
'Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador','Empacotador',
'Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja',
'Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja','Supervisor de Loja',
'Analista de TI','Analista de TI','Analista de TI','Analista de TI','Analista de TI',
'Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH',
'Analista de RH','Analista de RH','Analista de RH','Analista de RH','Analista de RH',
'Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro','Analista financeiro',
'Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo',
'Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo','Assistente Administrativo',
'Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque',
'Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque',
'Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque','Encarregado de estoque',
'Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente',
'Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente',
'Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente','Atendente',
'Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança',
'Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança','Segurança',
'Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro','Açougueiro',
'Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro','Padeiro',
'Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral',
'Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral',
'Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral','Ajudante geral', 'Ajudante geral',
'Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza',
'Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza','Auxiliar de limpeza',
'Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas',
'Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas','Promotor de vendas'
]

cargos1 = [
    'Repositor','Operador de caixa','Empacotador','Atendente','Ajudante geral','Auxiliar de limpeza']
cargos2 = [
    'Encarregado de estoque','Segurança','Padeiro','Açougueiro','Assistente Administrativo']
cargos3 = [
    'Supervisor de Loja','Promotor de vendas']
cargos4 = [
    'Analista de TI','Analista de RH','Analista financeiro']


cidades_brasil = [ 
    "Sao Paulo", "Rio de Janeiro", "Brasilia", "Salvador", "Fortaleza",
    "Belo Horizonte", "Manaus", "Curitiba", "Recife", "Goiania",
    "Porto Alegre", "Guarulhos", "Campinas", "Sao Luis",
    "Sao Goncalo", "Maceio", "Duque de Caxias", "Natal", "Teresina",
    "Sao Bernardo do Campo", "Joao Pessoa", "Osasco",
    "Jaboatao dos Guararapes", "Ribeirao Preto", "Uberlandia", "Contagem", "Sorocaba",
    "Aracaju", "Feira de Santana", "Cuiaba", "Joinville", "Aparecida de Goiania",
    "Londrina", "Juiz de Fora", "Ananindeua", "Belford Roxo", "Niteroi",
    "Sao Joao de Meriti", "Campos dos Goytacazes", "Serra", "Caxias do Sul", "Maua",
    "Macapa", "Piracicaba", "Canoas", "Bauru", "Sao Jose dos Campos"]

df_maiores_cidades = pd.read_sql("SELECT IDCIDADE FROM CIDADE WHERE NOME IN ({})".format(','.join(f"'{cidade}'" for cidade in cidades_brasil)), engine)['IDCIDADE'].tolist()

# Função para gerar um nome completo, gênero e e-mail
def gerar_nome_completo():
    nome = random.choice(nomes)
    sobrenome = random.choice(sobrenomes)
    sobrenome2 = random.choice(sobrenomes)
    sobrenome3 = random.choice(sobrenomes)
    nomecompleto = f'{nome} {sobrenome} {sobrenome2} {sobrenome3}'
    cargo = random.choice(cargos)
    if cargo in cargos1:
        salario = round(random.uniform(1550,1975),2)
    elif cargo in cargos2:
        salario = round(random.uniform(1815,2380),2)
    elif cargo in cargos3:
        salario = round(random.uniform(2472,3257),2)
    else:
        salario = round(random.uniform(3441,4115),2)
    rua = random.choice(nomes_ruas)
    num = random.randint(5, 5000)
    bairro = random.choice(nomes_bairros)
    endereco = f"{rua} - {num} - {bairro}"
    id_cidade = random.choice(df_maiores_cidades)
    id_loja = random.randint(1,150)
    id_gerente = random.randint(1,280)
    if cargo in ['Repositor','Operador de caixa','Empacotador','Encarregado de estoque','Segurança','Açougueiro','Padeiro','Ajudante geral','Auxiliar de limpeza']:
        id_departamento = 4
    elif cargo in ['Supervisor de Loja','Assistente Administrativo']:
        id_departamento = 1
    elif cargo in ['Atendente','Promotor de vendas']:
        id_departamento = 2
    elif cargo in ['Analista financeiro']:
        id_departamento = 3
    elif cargo in ['Analista de RH']:
        id_departamento = 5
    else:
        id_departamento = 6
    
    variavelcontratacao = random.randint(1,1000)
    fake = Faker() #variavel paaara usar Faker
    if variavelcontratacao <=865:
        situacao = 'Ativa'
        data_admissao = fake.date_time_between(start_date='-15y',end_date='now')
        data_demissao = ""
        data_criacao = data_admissao
        data_atlz = ""
        versao = 1
    else:
        situacao = 'Inativa'
        data_demissao = fake.date_time_between(start_date='-14y',end_date='now')
        data_admissao = fake.date_time_between(start_date='-15y',end_date=data_demissao)
        data_criacao = data_admissao
        data_atlz = data_demissao
        versao = 2
    
    return nomecompleto, cargo, endereco, salario, id_cidade, id_loja, id_gerente, id_departamento,situacao, data_admissao, data_demissao, data_criacao, data_atlz, versao

# Gerar 1000 nomes completos com gênero e e-mail
dados_nomes = [gerar_nome_completo() for _ in range(quant_funcionarios_gerados)]
df_funcionarios = pd.DataFrame(dados_nomes, columns=['NOME', 'CARGO', 'ENDERECO','SALARIO', 'ID_CIDADE', 'ID_LOJA', 'ID_GERENTE', 'ID_DEPARTAMENTO','SITUACAO', 'DATA_ADMISSAO','DATA_DEMISSAO', 'DATA_CRIACAO', 'DATA_ATLZ', 'VERSAO'])

#Função para gerar números aleatórios únicos
def gerar_numeros_unicos(intervalo_inicio, intervalo_fim, quantidade):
    if quantidade > (intervalo_fim - intervalo_inicio + 1):
        raise ValueError("A quantidade de números únicos solicitada excede o intervalo disponível.")
    numeros_unicos = random.sample(range(intervalo_inicio, intervalo_fim + 1), quantidade)
    return numeros_unicos
#Valores de entrada
intervalo_inicio = 10000000000  # Início do intervalo
intervalo_fim =    99999999999  # Fim do intervalo
quantidade_registros = quant_funcionarios_gerados  # Quantidade de registros no DataFrame

lista_cpf = gerar_numeros_unicos(intervalo_inicio, intervalo_fim, quantidade_registros)

df_funcionarios["CPF"] = lista_cpf

display(df_funcionarios)



Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida


,NOME,CARGO,ENDERECO,SALARIO,ID_CIDADE,ID_LOJA,ID_GERENTE,ID_DEPARTAMENTO,SITUACAO,DATA_ADMISSAO,DATA_DEMISSAO,DATA_CRIACAO,DATA_ATLZ,VERSAO,CPF
0,Amanda Cavalcanti Costa Nunes,Supervisor de Loja,Rua Santa Catarina - 1643 - Vila Nova Esperanca,3052.05,3828,3,246,1,Ativa,2020-07-06 21:47:33,,2020-07-06 21:47:33,,1,77893654954
1,Gustavo Rocha Fernandes Farias,Repositor,Rua Ceara - 3805 - Santa Helena,1883.60,3823,38,8,4,Ativa,2010-06-28 10:22:34,,2010-06-28 10:22:34,,1,74750139101
2,Luis Fonseca Barros Carvalho,Assistente Administrativo,Rua 7 de Setembro - 3859 - Santa Cecilia,2090.12,5214,44,129,1,Ativa,2016-06-29 19:58:46,,2016-06-29 19:58:46,,1,59847784061
3,Victor Nascimento Ferreira Dias,Repositor,Rua Monte Castelo - 4248 - Parque das Nacoes,1765.95,3066,125,39,4,Ativa,2019-10-18 01:28:31,,2019-10-18 01:28:31,,1,32191475448
4,Juliana Peixoto Fonseca Fernandes,Atendente,Rua do Porto - 4656 - Santa Angela,1571.86,3828,73,137,2,Ativa,2017-05-23 02:33:22,,2017-05-23 02:33:22,,1,54862778538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Marcela Santos Barros Santos,Operador de caixa,Rua dos Pinheiros - 532 - Jardim das Tulipas,1562.93,5214,7,35,4,Inativa,2013-05-02 02:34:05,2016-11-30 23:42:55,2013-05-02 02:34:05,2016-11-30 23:42:55,2,63124974456
496,Joao Barros Brito Campos,Encarregado de estoque,Rua Santos Dumont - 4065 - Jardim Santa Rita,2005.77,5336,44,129,4,Ativa,2012-05-05 11:21:30,,2012-05-05 11:21:30,,1,81309844551
497,Patricia Freitas Monteiro Costa,Auxiliar de limpeza,Rua Santa Monica - 2484 - Jardim Canada,1625.93,2662,111,53,4,Ativa,2017-11-16 08:35:39,,2017-11-16 08:35:39,,1,54502498775
498,Enzo Barros Castro Nunes,Repositor,Rua das Violetas - 3845 - Jardim Primavera,1728.38,3248,30,31,4,Ativa,2023-10-17 21:12:05,,2023-10-17 21:12:05,,1,83741937768
